In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
plt.rcParams['font.family'] = 'NanumGothic'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../01.Data_Collection/films_updated.csv')

In [3]:
df.shape

(687, 38)

In [4]:
df.isnull().sum()

영화명           0
영화코드          0
개봉일           0
대표국적          0
국적            0
장르            0
등급            0
영화형태          0
영화구분          0
제작사          17
배급사           0
상영횟수          0
전국스크린수        0
전국매출액         0
전국관객수         0
서울매출액         0
서울관객수         0
F-rated       0
감독            0
대표감독          0
감독코드          0
감독성별          0
감독_대표역할       0
감독필모          0
작가           42
대표작가         42
대표작가(필명)     42
작가코드         42
작가성별         68
작가_대표역할      42
작가필모         42
배우            0
주연배우          0
배역명         166
배우코드          0
배우성별          0
배우_대표역할       0
배우필모          0
dtype: int64

In [5]:
df[df['작가성별'].isnull()]['영화명']

179                     해어화
188              극장판 안녕 자두야
198                달에 부는 바람
203               소주와 아이스크림
204                  과대망상자들
212                      사냥
213                  빅뱅 메이드
214                      경계
215                  홀리워킹데이
217       극장판 프리즘스톤 올스타 셀렉션
234            브링 홈: 아버지의 땅
246                할머니의 먼 집
247                     돈 맛
257            무현, 두 도시 이야기
265           제자도 - 제자,옥한흠2
274               나의 살던 고향은
286                     위켄즈
295        터닝메카드W: 블랙미러의 부활
297        뚜르: 내 생애 최고의 49일
304                   레이디버그
311               비정규직 특수요원
322                    더 플랜
323               마리안느와 마가렛
326           서서평, 천천히 평온하게
337          고려 아리랑: 천산의 디바
349        청춘 합창단 - 또 하나의 꿈
358                  파밍 보이즈
369             밤섬해적단 서울불바다
372    무현, 두 도시 이야기 : 파이널 컷
373                     소나기
375                  저수지 게임
376                  안녕 히어로
385                    땐뽀걸즈
386              다시 태어나도 우리
404                  올드마린보이
422         뽀로로 극장판 

In [6]:
df['F-rated'].value_counts()

0    357
1    228
2     60
3     42
Name: F-rated, dtype: int64

In [7]:
# F-rated 다시 구하기
f_rated = []
for i, row in df.iterrows():
    answer = 0
    if row['배우성별'] == '여자' : answer += 1
    if row['작가성별'] == '여자' : answer += 1
    if row['감독성별'] == '여자' : answer += 1
    f_rated.append(answer)
df['F-rated'] = f_rated

In [8]:
df['F-rated'].value_counts()

0    354
1    228
2     61
3     44
Name: F-rated, dtype: int64

In [10]:
# csv 파일에 F-rated 업데이트
# df.to_csv('../01.Data_Collection/films_updated.csv', index=False)

In [11]:
df = pd.read_csv('../01.Data_Collection/films_updated.csv')

1. 대형 배급사의  F-rated 영화 배급 비율
2. 회당 매출액 [매출액/상영횟수? 관객수/상영횟수?] 로 구한 지표와 F-rated 등급을 포함한 다른 컬럼들과의 상관관계 
3. F-rated 등급과 상관계수 높은 피쳐가 무엇인지

In [ ]:
df.sample()

In [ ]:
# 

In [ ]:
df.groupby('F-rated')['전국매출액'].mean()

In [ ]:
df.groupby('F-rated')['전국매출액'].describe()

In [ ]:
plt.figure(figsize=[20,15])
sns.catplot(x='F-rated', y='전국매출액', data=df, kind='swarm')

In [ ]:
df.groupby('F-rated')['전국매출액'].median()

In [ ]:
df.groupby('F-rated')['상영횟수'].median()

In [ ]:
df['회당 평균 매출액'] = df['전국매출액'] / df['상영횟수']
df.groupby('F-rated')['회당 평균 매출액'].mean()

In [ ]:
df['회당 평균 매출액'] = df['전국매출액'] / df['상영횟수']
df.groupby('F-rated')['회당 평균 매출액'].median()

In [ ]:
df['스크린수 대비 매출액'] = df['전국매출액'] / df['전국스크린수']
df.groupby('F-rated')['스크린수 대비 매출액'].mean()

In [ ]:
df.groupby('F-rated')['스크린수 대비 매출액'].mean()

스크린마다 수용가능 관객수가 다르고 특정 영화만 대형관에 상영되는 것이 문제임은 어떻게 말하지?  
-> 각 영화가 어떤 스크린에서 상영했는지에 대한 데이터, 스크린별 수용인원 데이터 추가로 필요

In [ ]:
df.groupby('F-rated')['전국관객수'].mean()

In [ ]:
df['상영횟수 대비 관객수'] = df['전국관객수'] / df['상영횟수']
df.groupby('F-rated')['상영횟수 대비 관객수'].mean()

In [ ]:
df['스크린수 대비 관객수'] = df['전국관객수'] / df['전국스크린수']
df.groupby('F-rated')['스크린수 대비 관객수'].mean()